In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="mk0KG60ShShIw9dOeJZW")
project = rf.workspace("kyushu-university-dwxf1").project("fyp_20230907")
dataset = project.version(1).download("yolov8")


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 63.3/63.3 kB 3.5 MB/s eta 0:00:00
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
     ---------------------------------------- 38.0/38.0 MB 9.6 MB/s eta 0:00:00
     ---------------------------------------- 67.8/67.8 kB ? eta 0:00:00
     ---------------------------------------- 72.2/72.2 kB 4.1 MB/s eta 0:00:00
  Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)
     ---------------------------------------- 54.5/54.5 kB ? eta 0:00:00
     ---------------------------------------- 7.6/7.6 MB 11.4 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 11.1 MB/s eta 0:00:00
     ------------------------------------- 181.9/181.9 kB 10.7 MB/s eta 0:00:00


C:\Users\LENOVO\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.



Extracting Dataset Version Zip to fyp_20230907-1 in yolov8:: 100%|██████████████████| 289/289 [00:00<00:00, 683.47it/s]


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
# Define parameters
batch_size = 16
image_size = (224, 224)
# Define the dataset path
train_data_dir = 'C:/Users/LENOVO/Desktop/Untitled Folder/fyp_20230907-1/train/'
valid_data_dir = 'C:/Users/LENOVO/Desktop/Untitled Folder/fyp_20230907-1/valid/'

In [12]:
# Create ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [13]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)



Found 98 images belonging to 13 classes.


In [15]:
valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',

)

Found 42 images belonging to 13 classes.


In [16]:
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")

Number of classes: 13


In [17]:
# Build a simple CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

In [18]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=100  # you can adjust the number of epochs
)

Epoch 1/100
6/6 [==============================] - 11s 1s/step - loss: 3.9348 - accuracy: 0.0610 - val_loss: 2.5276 - val_accuracy: 0.0938
Epoch 2/100
6/6 [==============================] - 6s 1s/step - loss: 2.6083 - accuracy: 0.0488 - val_loss: 2.4991 - val_accuracy: 0.2188
Epoch 3/100
6/6 [==============================] - 6s 995ms/step - loss: 2.5535 - accuracy: 0.0854 - val_loss: 2.4398 - val_accuracy: 0.2500
Epoch 4/100
6/6 [==============================] - 6s 1s/step - loss: 2.4246 - accuracy: 0.1829 - val_loss: 2.2503 - val_accuracy: 0.1562
Epoch 5/100
6/6 [==============================] - 6s 1s/step - loss: 2.2126 - accuracy: 0.1829 - val_loss: 1.8632 - val_accuracy: 0.3438
Epoch 6/100
6/6 [==============================] - 7s 1s/step - loss: 1.9410 - accuracy: 0.3171 - val_loss: 1.8253 - val_accuracy: 0.3438
Epoch 7/100
6/6 [==============================] - 7s 1s/step - loss: 1.6465 - accuracy: 0.4688 - val_loss: 1.6920 - val_accuracy: 0.4062
Epoch 8/100
6/6 [=============

6/6 [==============================] - 6s 1s/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 1.5416 - val_accuracy: 0.7500
Epoch 60/100
6/6 [==============================] - 6s 935ms/step - loss: 0.0224 - accuracy: 0.9878 - val_loss: 1.2783 - val_accuracy: 0.8125
Epoch 61/100
6/6 [==============================] - 6s 982ms/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 0.8303 - val_accuracy: 0.8750
Epoch 62/100
6/6 [==============================] - 6s 1s/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 1.2837 - val_accuracy: 0.8125
Epoch 63/100
6/6 [==============================] - 6s 1s/step - loss: 0.0786 - accuracy: 0.9688 - val_loss: 0.9454 - val_accuracy: 0.8125
Epoch 64/100
6/6 [==============================] - 6s 863ms/step - loss: 0.0706 - accuracy: 0.9878 - val_loss: 3.5710 - val_accuracy: 0.6562
Epoch 65/100
6/6 [==============================] - 6s 931ms/step - loss: 0.3984 - accuracy: 0.9024 - val_loss: 1.4486 - val_accuracy: 0.6875
Epoch 66/100
6/6 [==========

In [20]:
# Save the model
model.save('origami_shape_classifier.h5')

In [40]:
import tensorflow as tf
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('origami_shape_classifier.h5')


In [51]:

# Path to your test image
test_image_path = 'C:/Users/LENOVO/Downloads/hqdefault.jpg'


In [52]:

# Load and preprocess the test image
img = tf.keras.preprocessing.image.load_img(test_image_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the image


In [53]:

# Make predictions
predictions = model.predict(img_array)


1/1 [==============================] - 0s 96ms/step


In [54]:

# Map predicted class index to the actual class name
class_names = train_generator.class_indices
predicted_class = np.argmax(predictions)
predicted_class_name = [k for k, v in class_names.items() if v == predicted_class][0]


In [55]:

print(f"Predicted class index: {predicted_class}")
print(f"Predicted class name: {predicted_class_name}")


Predicted class index: 6
Predicted class name: rabbit
